# Installation Instructions

Clone the repos, install dependencies and install packages terrapin, quest and gsshapy.

Note: terrapin isn't used in this example but it will be in future examples...

```bash
git clone git@public.git.erdc.dren.mil:computational-analysis-and-mechanics/quest.git
git clone git@public.git.erdc.dren.mil:computational-analysis-and-mechanics/terrapin.git
git clone git@github.com:CI-WATER/gsshapy.git

conda env create -n earthsim -f ./terrapin/py3_conda_environment.yml
conda env update -n earthsim -f ./quest/py3_conda_environment.yml
conda env update -n earthsim -f ./gsshapy/conda_env.yml
source activate earthsim

cd terrapin
python setup.py develop
cd ../quest
python setup.py develop
cd ../gsshapy
python setup.py develop
```


In [1]:
%matplotlib inline
from datetime import datetime, timedelta
import os

import geopandas as gpd
from gsshapy.modeling import GSSHAModel
import quest
from matplotlib import colors, cm, pyplot as plt
from gazar import grid
import numpy as np

ModuleNotFoundError: No module named 'gsshapy'

In [ ]:
VICKSBURG = 'vicksburg'
PROVO = 'provo'

location = PROVO
# location = VICKSBURG

BBOXES = {VICKSBURG: [-90.9, 32.2, -90.8, 32.3],
          PROVO: [-111.6, 40.0, -111.4, 40.15]
         }

OUTLETS = {VICKSBURG: (-90.885, 32.2158), # (-90.889, 32.2133) Move point out of Mississippi River,
           PROVO: (-111.44851851851851, 40.0787962962963),
          }

#service_feature = SERVICE_FEATURES[location]
bbox = BBOXES[location]
outlet = [OUTLETS[location]]

Function to automate the downloading process:

In [ ]:
def download_data(service_uri, bbox, collection):
    """
    Downloads raster data from source uri and adds to a quest collection.
    
    If multiple raster tiles are retrieved for the given bounds it calls a quest 
    filter to merge the tiles into a single raster.
    
    Returns quest uid for merged raster
    """
    
    features = quest.api.search_catalog(service_uri, filters={'bbox': bbox})
    print('Selected raster tiles:', features)
    features = quest.api.add_datasets(collection, features)
    datasets = quest.api.stage_for_download(features)
    print('start download -->')
    quest.api.download_datasets(datasets)
    print('start merge & clip -->')
    result = quest.api.run_tool('raster-merge', datasets=datasets, options={'bbox': bbox})
    merged_dataset = result['datasets'][0]
    print('merged raster:', merged_dataset)
    
    return merged_dataset

Parameters to change for the run:

In [ ]:
elevation_service = 'svc://usgs-ned:13-arc-second'
land_use_service = 'svc://usgs-nlcd:2011'
land_use_grid_id = 'nlcd'
base_dir = os.getcwd() # path needs to be absolute for gsshapy
gssha_model_name = location + '_with_delineation'
gssha_model_directory = os.path.join(base_dir, gssha_model_name)
boundary_shapefile = os.path.join(base_dir, gssha_model_name, 'watershed_boundary.shp')

The collection is where we will origanize the downloaded data.

In [ ]:
## Delete all collections
# for collection in quest.api.get_collections():
#   quest.api.delete(collection)

In [ ]:
try:
    quest.api.new_collection(gssha_model_name)
except ValueError as e:
    print(e)

Download the data. This may take some time.

In [ ]:
elevation = download_data(elevation_service, bbox, gssha_model_name)

In [ ]:
def watershed_delineation(elevation, outlet):
    # run pit filling algorithm
    algorithm = 'go-fill'  # one of ['flats', 'go-fill', 'go-breach']
    result = quest.api.run_tool('raster-fill', datasets=elevation, options={'algorithm': algorithm})
    pit_filled = result['datasets'][0]
    
    # run flow accumulation
    algorithm = 'go-d8'  # one of ['d8', 'go-d8', 'go-fd8']
    result = quest.api.run_tool('raster-flow-accumulation', datasets=pit_filled, options={'algorithm': algorithm})
    flow_accumulation = result['datasets'][0]
    print(flow_accumulation)
    
    # snap outlet point
    result = quest.api.run_tool('raster-watershed-snap-outlet', datasets=flow_accumulation, 
                       options={'outlet_points': outlet, 'snap_outlets': 'jenson', 'stream_threshold_pct': 0.01})
    snapped_outlet = result['features']['outlet']
    
    # generate delineation
    result = quest.api.run_tool('raster-watershed-delineation', datasets=pit_filled, features=[snapped_outlet])

    return result['features']['watershed'], result['features']['outlet'], flow_accumulation

In [ ]:
# use the merged raster to create a shapefile
watershed, outlet, flow_accumulation = watershed_delineation(elevation, outlet)
print(watershed, outlet, flow_accumulation)

In [ ]:
# viz watershed
watershed_geometry = quest.api.get_metadata(watershed)[watershed]['geometry']
outlet_lon, outlet_lat = np.array(quest.api.get_metadata(outlet)[outlet]['geometry'].coords.xy).squeeze().tolist()
print(watershed_geometry.area, outlet_lon, outlet_lat)
watershed_geometry

In [ ]:
## read in & plot flow accumulation
flow_accumulation_file = quest.api.get_metadata(flow_accumulation)[flow_accumulation]['file_path']
flow_accumulation_dataset = grid.GDALGrid(flow_accumulation_file)
flow_accumulation_data = flow_accumulation_dataset.np_array()

max_accumulation = flow_accumulation_data.max()
stream_threshold = max_accumulation * 0.01
rivers = np.ma.masked_where(flow_accumulation_data < stream_threshold, flow_accumulation_data)
plt.imshow(flow_accumulation_data, cmap=cm.terrain, norm=colors.LogNorm())

In [ ]:
# read in elevation data
dem_file = quest.api.get_metadata(elevation)[elevation]['file_path']
dem = grid.GDALGrid(dem_file)
lat, lon = dem.latlon

In [ ]:
# plot outlet pixel and river network
window = 200
stream_threshold = flow_accumulation_data.max() * 0.01
rivers = np.ma.masked_where(flow_accumulation_data < stream_threshold, flow_accumulation_data)
fig, ax = plt.subplots()
ax.imshow(np.flip(dem.np_array(), 0))
plt.imshow(np.flip(rivers, 0), cmap=cm.gray_r, norm=colors.LogNorm(), origin="lower")
outlet_col, outlet_row = flow_accumulation_dataset.lonlat2pixel(outlet_lon, outlet_lat)
ax.scatter(x=outlet_col, y=rivers.shape[0] - outlet_row, s=50, color='red')
ax.set_xlim((outlet_col-window,outlet_col+window))
ax.set_ylim((outlet_row-window,outlet_row+window))

In [ ]:
# plot the generated watershed with outlet point
watershed_geometry = quest.api.get_metadata(watershed)[watershed]['geometry']
outlet_lon, outlet_lat = quest.api.get_metadata(outlet)[outlet]['geometry'].coords.xy

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from matplotlib import colors, cm, pyplot as plt

xmin, ymin, xmax, ymax = bbox
display_bbox = [xmin, xmax, ymin, ymax]
graticule_spacing = 0.01

fig = plt.figure(figsize=(10, 10))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent(display_bbox)

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=2, color='gray', alpha=0.5, linestyle='--')
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabels_top = False

plt.contourf(lon, lat, dem.np_array(), 100, zorder=2, cmap=cm.terrain, transform=ccrs.PlateCarree())
ax.imshow(rivers, cmap=cm.ocean, origin='lower', zorder=3)
watershed_shp = cfeature.ShapelyFeature([watershed_geometry], ccrs.PlateCarree())
ax.add_feature(watershed_shp, zorder=4, alpha=0.6)
ax.scatter(x=outlet_lon, y=outlet_lat, color='red', zorder=5, transform=ccrs.PlateCarree())
plt.title("Watershed")
plt.show()

In [ ]:
land_use = download_data(land_use_service, bbox, gssha_model_name)

Use the data to generate a GSSHA model. This may take a minute.

In [ ]:
# make the directory for the output
try:
    os.mkdir(gssha_model_directory)
except OSError:
    pass

In [ ]:
# Save Watershed as a Shapefile for GSSHA
ws = quest.api.get_metadata(watershed, as_dataframe=True)

In [ ]:
# workaround for crs not being saved correctly
ws.crs = {'init' :'epsg:4326'}

In [ ]:
# Save Watershed as a Shapefile for GSSHA
del ws['created_at'] # gpd error on save
ws.to_file(boundary_shapefile)

In [ ]:
elevation_file_path = quest.api.get_metadata(elevation)[elevation]['file_path']
land_use_file_path = quest.api.get_metadata(land_use)[land_use]['file_path']

In [ ]:
# generate GSSHA model files
model = GSSHAModel(project_name=gssha_model_name,
                   project_directory=gssha_model_directory,
                   mask_shapefile=boundary_shapefile,
                   elevation_grid_path=elevation_file_path,
                   land_use_grid=land_use_file_path,
                   land_use_grid_id=land_use_grid_id,
                   out_hydrograph_write_frequency=1,
                   )

# add card for max depth
model.project_manager.setCard('FLOOD_GRID',
                              '{0}.fgd'.format(gssha_model_name),
                              add_quotes=True)
# TODO: Add depth grids to simulation
# MAP_FREQ, DEPTH

# add event for simulation
model.set_event(simulation_start=datetime.utcnow(),
                simulation_duration=timedelta(seconds=2*60),
                rain_intensity=24,
                rain_duration=timedelta(seconds=1*60),
                )
model.write()